In [1]:
import pandas as pd

In [23]:
# 1、获取数据

data = pd.read_csv('/Users/zhangqihao/Desktop/PYTHON/机器学习/机器学习资料2/02-代码/FBlocation/train.csv')

In [71]:
# 2、处理数据

# (1)因为有两千万行数据，为了方便起见，略微缩小数据范围
data = data.query("x < 2.5 & x > 2 & y < 1.5 & y > 1.0")

In [72]:
# (2)将时间戳转换为具体时间
time_value = pd.to_datetime(data.time, unit='s')
date = pd.DatetimeIndex(time_value)

# 选取有用的时间数据加入原数据中
data["day"] = date.day
data["weekday"] = date.weekday
data["hour"] = date.hour

In [73]:
# (3)过滤签到次数少的地点
place_count = data.groupby("place_id").count()["row_id"]

# 选择签到地点大于3次的地点
# place_count[place_count > 3]

# 选取原数据中的地点
# data[data~~~~]意思是Bool索引
data[data["place_id"].isin(place_count[place_count > 3].index.values)]

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9
1045,1045,2.3859,1.1660,498,503378,6438240873,6,1,19
...,...,...,...,...,...,...,...,...,...
29115112,29115112,2.1889,1.2914,168,721885,4606837364,9,4,8
29115204,29115204,2.1193,1.4692,58,563389,2074133146,7,2,12
29115338,29115338,2.0007,1.4852,25,765986,6691588909,9,4,20
29115464,29115464,2.4132,1.4237,61,151918,7396159924,2,4,18


In [74]:
# 筛选特征值和目标值

x = data[["x", "y", "accuracy", "day", "weekday", "hour"]]
y = data["place_id"]

In [75]:
# 数据集划分
from sklearn.model_selection import train_test_split

In [76]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [77]:
# 3、特征工程：标准化
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [78]:
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)

# 当对测试集进行标准化时，因为要与训练集一样的处理，所以无需fit()
x_test = transfer.transform(x_test)

In [79]:
# 4、 KNN算法预估器
estimator = KNeighborsClassifier(n_neighbors=5)

In [80]:
# 5、 网格搜索与交叉验证
param_dict = {"n_neighbors":[3,5,7,9]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)

In [81]:
# 6. KNN fit()
estimator.fit(x_train, y_train)

/Users/zhangqihao/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None, param_grid={'n_neighbors': [3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [82]:
# 7、模型预估

# 方法1:直接比对真实值和预测值
y_predict = estimator.predict(x_test)

print("y_predict:\n",y_predict)
print("直接比对真实值和预测值:\n", y_test == y_predict)

# 方法2:计算准确率
score = estimator.score(x_test, y_test)
print("准确率为:\n",score)

# 查看参数

print("最佳参数：/n", estimator.best_params_)
print("最佳结果：/n", estimator.best_score_)
print("最佳估计器：/n", estimator.best_estimator_)
print("交叉验证结果：/n", estimator.cv_results_)

y_predict:
 [4861093827 9936666116 6466855273 ... 2494678001 9081742495 6706708436]
直接比对真实值和预测值:
 7849873      True
11930876    False
17389925     True
11590281     True
19481529    False
            ...  
13568628     True
18800539     True
27764702    False
29084735    False
23362925     True
Name: place_id, Length: 20800, dtype: bool
准确率为:
 0.3546634615384615
最佳参数：/n {'n_neighbors': 7}
最佳结果：/n 0.3241021202942449
最佳估计器：/n KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')
交叉验证结果：/n {'mean_fit_time': array([0.02665567, 0.02709627, 0.02840535, 0.02681295]), 'std_fit_time': array([0.00042173, 0.00059693, 0.00081453, 0.00090985]), 'mean_score_time': array([0.74091236, 0.81891672, 0.89655256, 0.8960851 ]), 'std_score_time': array([0.02615845, 0.03884829, 0.07696012, 0.01698747]), 'param_n_neighbors': masked_array(data=[3, 5, 7, 9],
             mask=[Fals